# Librerías

In [ ]:
import sys
import os
sys.path.append(".")
import re
import copy
import pandas as pd
import numpy as np
import pandasql as ps
from datetime import timedelta
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import seaborn as sns

color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', None)
def thousands_formatter_func(x, pos):
    return f'{int(x / 1e3)}K'
thousand_formatter = FuncFormatter(thousands_formatter_func)
def decimal_percentage_formatter_func(x, pos):
    return f'{int(x * 100)}%'
decimal_percentage_formatter = FuncFormatter(decimal_percentage_formatter_func)
def percentage_formatter_func(x, pos):
    return f'{int(x)}%'
percentage_formatter = FuncFormatter(percentage_formatter_func)

# Lectura archivos

In [ ]:
df_bombs = pd.read_csv('data/processed_csv/df_bombs.csv', delimiter=',', encoding='latin-1', index_col=0)
df_of = pd.read_csv('data/processed_csv/df_of.csv', delimiter=',', encoding='latin-1')
df_operators = pd.read_csv('data/processed_csv/df_operators.csv', delimiter=',', encoding='latin-1')
df_operators_participation = pd.read_csv('data/processed_csv/df_operators_participation.csv', delimiter=',', encoding='latin-1')

df_bombs['start_date'] = pd.to_datetime(df_bombs['start_date'], format='%Y-%m-%d %H:%M:%S.%f')
df_bombs['end_date'] = pd.to_datetime(df_bombs['end_date'], format='%Y-%m-%d %H:%M:%S.%f')
df_of['start_date'] = pd.to_datetime(df_of['start_date'], format='%Y-%m-%d %H:%M:%S.%f')
df_of['end_date'] = pd.to_datetime(df_of['end_date'], format='%Y-%m-%d %H:%M:%S.%f')



In [ ]:
df_of.line = df_of.line.replace({"LÃ\x8dNEA 2": "LINEA_2", "LINEA_4": "LINEA_4", "LÃ\x8dNEA 1": "LINEA_1", "LINEA_6": "LINEA_6",
                    "LINEA 3": "LINEA_3", "LÃ\x8dNEA KIVU": "LINEA_KIVU", "PREFILTRO L-1": "PREFILTRO_L-1", "PREFILTRO L-6": "PREFILTRO_L-6",
                        "LINEA 7": "LINEA_7", "LINEA 8 IML": "LINEA_8_IML"})
df_operators_participation.line = df_operators_participation.line.replace({"LÃ\x8dNEA 2": "LINEA_2", "LINEA_4": "LINEA_4", "LÃ\x8dNEA 1": "LINEA_1", "LINEA_6": "LINEA_6",
                    "LINEA 3": "LINEA_3", "LÃ\x8dNEA KIVU": "LINEA_KIVU", "PREFILTRO L-1": "PREFILTRO_L-1", "PREFILTRO L-6": "PREFILTRO_L-6",
                        "LINEA 7": "LINEA_7", "LINEA 8 IML": "LINEA_8_IML"})

df_of.line.unique()

In [ ]:
def remove_special_chars(text):
    # Agrega los caracteres especiales que deseas eliminar, incluyendo los tildes
    special_chars = r"[^\w\sáéíóúÁÉÍÓÚñÑÃ]"
    text_without_special_chars = re.sub(special_chars, '', text)
    # Agrega aquí cualquier otro reemplazo adicional que desees realizar
    return text_without_special_chars

In [ ]:
df_of['line'].apply(remove_special_chars)

In [ ]:
df_of["weekday"] = df_of["start_date"].dt.weekday.astype("category")
df_of["turn"] = df_of["start_date"].apply(lambda x: 'AM' if x.hour < 14 else 'PM')
df_of["month"] = df_of["start_date"].dt.month.astype("category")
df_of["year"] = df_of["start_date"].dt.year.astype("category")


In [ ]:
df_operators_participation.head()

# Cargar modelo

In [ ]:
import pickle

# Cargar el modelo desde el archivo
with open('classifier_model.pickle', 'rb') as file:
    classifier_model = pickle.load(file)

# Algoritmo

In [ ]:
import itertools
import numpy as np
import pandas as pd

def select_best_combo(orders, available_operators, available_lines, max_operators_per_order, model):
    """
    Itera sobre todas las combinaciones posibles de operadores y lineas y selecciona la que tiene
    la mejor predicción para todas las OFs, con la restricción de que un operador no puede trabajar
    en más de una OF a la vez.
    """
    best_prediction = np.inf  # inicializar con infinito para minimizar
    best_order_operator_line = {}

    # Creamos todas las combinaciones de operadores y líneas
    operator_combos = []
    for r in range(1, max_operators_per_order + 1):
        operator_combos.extend(itertools.combinations(available_operators, r))
    operator_line_combos = list(itertools.product(operator_combos, available_lines))

    # Recorremos todas las órdenes
    for order in orders:
        order_best_prediction = np.inf
        order_best_operator_line = None

        # Probamos cada combinación de operador-línea para la orden
        for operators, line in operator_line_combos:
            # Creamos un dataframe con las características de la orden
            order_df = pd.DataFrame({
                'good_qty': [order['good_qty']],
                'theorical_time': [order['theorical_time']],
                'operators': [operators],
                'line': [line],
            })

            # Hacemos la predicción con el modelo
            prediction = model.predict(order_df)[0]

            # Si la predicción es la mejor hasta ahora para esta orden, la guardamos
            if prediction < order_best_prediction:
                order_best_prediction = prediction
                order_best_operator_line = (operators, line)

        # Almacenamos la mejor combinación operador-línea para esta orden
        best_order_operator_line[order['id']] = order_best_operator_line

        # Actualizamos la mejor predicción general
        if order_best_prediction < best_prediction:
            best_prediction = order_best_prediction

        # Eliminamos la combinación de operador-línea seleccionada de la lista de combinaciones
        operator_line_combos = [combo for combo in operator_line_combos if not set(order_best_operator_line[0]).issubset(set(combo[0]))]

    return best_order_operator_line, best_prediction


In [ ]:
df_operators_participation.head()

In [ ]:
ORDERS = [
    {'id': 5160396, 'good_qty': 100, 'theorical_time': 2.5, 'registers_qty': 30, 'operators_distinct_qty': 2, 'days_accumulated_experience': 102, 'OFs_accumulated_experience': 50},
    {'id': 5169247, 'good_qty': 200, 'theorical_time': 3.0, 'registers_qty': 10, 'operators_distinct_qty': 3, 'days_accumulated_experience': 102, 'OFs_accumulated_experience': 50},
    {'id': 5171973, 'good_qty': 150, 'theorical_time': 2.0, 'registers_qty': 50, 'operators_distinct_qty': 4, 'days_accumulated_experience': 102, 'OFs_accumulated_experience': 50},
]

available_operators = [1, 2, 3, 4, 5, 6, 7, 8, 9]
available_lines = ['LINEA_1', 'LINEA_2', 'LINEA_3', 'LINEA_4']


In [ ]:
from deap import creator, base, tools, algorithms
import random
import itertools

In [ ]:
def format_df_to_classifier_model(order, operators, line, available_operators, available_lines):
    # DataFrame de ceros con las columnas que necesitamos
    columns = ['good_qty', 'theorical_time', 'registers_qty', 'operators_distinct_qty'] + ['operator_' + str(i) for i in available_operators] + ['line_' + line for line in available_lines] + ['days_accumulated_experience', 'OFs_accumulated_experience']
    order_df = pd.DataFrame(np.zeros((1, len(columns))), columns=columns)
    # Actualiza las columnas relevantes
    order_df['good_qty'] = order['good_qty']
    order_df['theorical_time'] = order['theorical_time']
    order_df['registers_qty'] = order['registers_qty']
    order_df['operators_distinct_qty'] = len(operators)
    order_df['days_accumulated_experience'] = order['days_accumulated_experience']
    order_df['OFs_accumulated_experience'] = order['OFs_accumulated_experience']
    for operator in operators:
        order_df['operator_' + str(operator)] = 1

    order_df['line_' + line] = 1
    return order_df

In [ ]:
unique_individuals = set()
evaluations_count = 0
def individual_to_str(individual):
    return str(sorted((assignment['order'], tuple(sorted(assignment['operators'])), assignment['line']) for assignment in individual))

def evaluate(individual):
    global evaluations_count, unique_individuals
    evaluations_count += 1
    # Agrega el individuo al conjunto de individuos únicos
    unique_individuals.add(individual_to_str(individual))

    success_count = 0
    total_probability = 0.0
    for assignment in individual:

        order_id = assignment['order']
        operators = assignment['operators']
        line = assignment['line']

        # Encuentra la orden correspondiente en la lista de órdenes
        order = next(order for order in ORDERS if order['id'] == order_id)
        
        order_df = pd.DataFrame({
            'good_qty': [order['good_qty']],
            'theorical_time': [order['theorical_time']],
            'operators': [operators],
            'line': [line],
        })
        format_df_to_classifier_model(order, operators, line, available_operators, available_lines)
        # prediction = classifier_model.predict(order_df)[0]
        prediction_proba = random.random()
        prediction = 1 if prediction_proba > 0.5 else 0

        if prediction == 1:  # Si la OF es exitosa
            success_count += 1
        total_probability += prediction_proba
    
    fitness = success_count + (total_probability / 10000) 
    return fitness,

operator_combos = []
max_operators_per_order = 4
for r in range(1, max_operators_per_order + 1):
    operator_combos.extend(itertools.combinations(available_operators, r))
operator_line_combos = list(itertools.product(operator_combos, available_lines))

# Definimos los tipos para el problema de optimización
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # Maximizamos la función de fitness
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# La función initIndividual crea un individuo, que es una lista de asignaciones de operadores y líneas a las órdenes
def initIndividual():
    individual = []
    available_operators_for_this_individual = available_operators.copy()  # Crear una copia de la lista de operadores disponibles
    available_lines_for_this_individual = available_lines.copy()  # Crear una copia de la lista de líneas disponibles
    for order in ORDERS:
        # Elegir un conjunto de operadores que aún no se hayan asignado
        valid_operator_combos = [combo for combo in operator_combos if set(combo).issubset(available_operators_for_this_individual)]
        if not valid_operator_combos:  # Si no hay más operadores disponibles, no podemos generar un individuo válido
            return None
        operators = random.choice(valid_operator_combos)
        for operator in operators:
            available_operators_for_this_individual.remove(operator)  # El operador ya no está disponible para las siguientes órdenes

        # Elegir una línea que aún no se haya asignado
        if not available_lines_for_this_individual:  # Si no hay más líneas disponibles, no podemos generar un individuo válido
            return None
        line = random.choice(available_lines_for_this_individual)
        available_lines_for_this_individual.remove(line)  # La línea ya no está disponible para las siguientes órdenes

        individual.append({'order': order['id'], 'operators': operators, 'line': line})
    print('[INIT INDIVIDUAL]:')
    print(individual)  
    return creator.Individual(individual)


toolbox.register("individual", initIndividual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Definimos los operadores genéticos
toolbox.register("evaluate", evaluate)


# Definimos la función de crossovr
def custom_crossover(ind1, ind2):
    # Crea copias de los individuos para no modificar los originales
    new_ind1 = copy.deepcopy(ind1)
    new_ind2 = copy.deepcopy(ind2)
    
    for order in ORDERS:
        order_id = order['id']

        # Encuentra las asignaciones para esta orden en ambos individuos
        assignment1 = next(assignment for assignment in new_ind1 if assignment['order'] == order_id)
        assignment2 = next(assignment for assignment in new_ind2 if assignment['order'] == order_id)
        
        # Intercambia las asignaciones de operadores y líneas
        assignment1['operators'], assignment2['operators'] = assignment2['operators'], assignment1['operators']
        assignment1['line'], assignment2['line'] = assignment2['line'], assignment1['line']


    # Verifica si las nuevas asignaciones violan las restricciones
    violation = False
    for new_ind in [new_ind1, new_ind2]:
        for i in range(len(new_ind1)):
            for j, assignment in enumerate(new_ind):
                if j != i:
                    # Verifica si la nueva línea se repite en otras asignaciones
                    if new_ind[i]['line'] == assignment['line']:
                        violation = True
                        break

                    # Verifica si los nuevos operadores se repiten en otras asignaciones
                    if any(op in new_ind[i]['operators'] for op in assignment['operators']):
                        violation = True
                        break
    if violation:
        print('[ERROR-CROSSOVER VIOLATION]:')
        print(new_ind1)
        print(new_ind2)
        return ind1, ind2
    
    print('[CROSSOVER GENERATED]')
    return new_ind1, new_ind2

toolbox.register("mate", custom_crossover)

def custom_mutation(individual, indpb):
    attempts = 0
    for i in range(len(individual)):
        if random.random() < indpb:
            attempts = 0
            max_attempts = 10000  # ajusta este número según sea necesario
            while attempts < max_attempts:
                # Selecciona una nueva línea que aún no se haya asignado
                new_line = random.choice(available_lines)

                # Verifica si la nueva asignación viola las restricciones
                violation = False
                new_operators = random.choice(operator_combos)
                for j, assignment in enumerate(individual):
                    if j != i:
                        # Verifica si la nueva línea se repite en otras asignaciones
                        if new_line == assignment['line']:
                            violation = True
                            break

                        # Verifica si los nuevos operadores se repiten en otras asignaciones
                        if any(op in new_operators for op in assignment['operators']):
                            violation = True
                            break

                if not violation:
                    # Si la asignación es válida, actualiza la asignación en el individuo
                    individual[i]['line'] = new_line
                    individual[i]['operators'] = tuple(new_operators)  # Convierte la lista en una tupla
                    break

                attempts += 1
    print(f'[MUTATION GENERATED]: {attempts or 0} intentos')
    return individual,

toolbox.register("mutate", custom_mutation, indpb=0.4)
toolbox.register("select", tools.selTournament, tournsize=3)

# Inicializamos la población y ejecutamos el algoritmo genético
population = toolbox.population(n=50)
result = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, verbose=False)
best_individual = tools.selBest(result[0], 3)[0]
best_fitness = best_individual.fitness.values

print('···· RESULTADOS ····')
print(f'Se han realizado {evaluations_count} evaluaciones.')
# print(f'Se han realizado {len(unique_individuals)} evaluaciones únicas.')
print(f'El mejor individuo es: {best_individual}')
def selectUnique(individuals, k):
    unique_individuals = []
    for ind in individuals:
        if not any(ind2 == ind for ind2 in unique_individuals):
            unique_individuals.append(ind)
        if len(unique_individuals) == k:
            break
    return unique_individuals

unique_individuals = selectUnique(tools.selNSGA2(result[0], len(result[0])), 10)
best_fitnesses = [ind.fitness.values for ind in unique_individuals]

for i in range(len(unique_individuals)):
    print('########################')
    print(f'\nMejor individuo {i+1}:')
    print(f'Valor de fitness: {best_fitnesses[i]}')
    for assignment in unique_individuals[i]:
        order_id = assignment['order']
        operators = assignment['operators']
        line = assignment['line']
        print(f'  Orden: {order_id}, Operadores: {operators}, Línea: {line}')


In [ ]:
example = [{'order': 5160396, 'operators': (1, 2, 9), 'line': 'LINEA_3'}, {'order': 5169247, 'operators': (3, 4), 'line': 'LINEA_2'}, {'order': 5171973, 'operators': (3,), 'line': 'LINEA_4'}]

In [ ]:
for i in range(len(example)):
    violation = False
    for j, assignment in enumerate(example):
        if j != i:
            # Verifica si la nueva línea se repite en otras asignaciones
            if example[i]['line'] == assignment['line']:
                violation = True
                break

            # Verifica si los nuevos operadores se repiten en otras asignaciones
            if any(op in example[i]['operators'] for op in assignment['operators']):
                violation = True
                break
print(violation)